In [1]:
import pandas as pd
import numpy as np
import requests
import time
import pandas as pd
from sqlalchemy import create_engine

#import info from config file
from config import weather_api_key
from config import db_pass

# Extract

In [19]:
#read winemag-data-130k-v2.csv from resources
wine_file = "Resources/winemag-data-130k-v2.csv"
wine_df = pd.read_csv(wine_file)
wine_df.head()

,id,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [ ]:
# city_weather = []

# #variables for records
# record_num = 0
# Set = 1

# print("Beginning Data Retrieval")
# print("----------------------------")

# #loop through cities retreiving data for each region
# for index, row in provinces_cleaned.iterrows():
#     time.sleep(1)
    
#     #check to see if there is data for region
#     try:
#         i = row["region_1"]
#         query_url = (f"https://api.openweathermap.org/data/2.5/weather?q={i}&appid={weather_api_key}&units=imperial")
#         response = requests.get(query_url).json()

#         if record_num == 50:
#             Set += 1
#             record_num = 0
#         else:
#             record_num += 1 
            
#         #print record saying which region is being processed
#         print(f"Processing Record {record_num} of Set {Set} | {i}")
        
#         #get data for region 
#         region_lng = response['coord']['lon']
#         region_lat = response['coord']["lat"]
#         region_temp= response['main']['temp']
#         region_max_temp = response['main']['temp_max']
#         region_min_temp = response['main']['temp_min']
#         region_humid = response['main']['humidity']
#         region_country = response['sys']['country']
  

#         #Create dictionary with data gathered
#         city_weather.append({
#         "Region": i,
#         "Lat": region_lat,
#         "Lng": region_lng,
#         "Temp": region_temp,
#         "Max Temp": region_max_temp,
#         "Min Temp": region_min_temp,
#         "Humidity": region_humid
#         })

#     except:
#         print(f"region {i} not found... Sorry!")

#         try:
#             a = row["province"]
#             query_url = (f"https://api.openweathermap.org/data/2.5/weather?q={a}&appid={weather_api_key}&units=imperial")
#             response = requests.get(query_url).json()

#             #get data for region 
#             region_lng = response['coord']['lon']
#             region_lat = response['coord']["lat"]
#             region_temp= response['main']['temp']
#             region_max_temp = response['main']['temp_max']
#             region_min_temp = response['main']['temp_min']
#             region_humid = response['main']['humidity']
            


#             #Create dictionary with data gathered
#             city_weather.append({
#             "Region": i,
#             "Lat": region_lat,
#             "Lng": region_lng,
#             "Temp": region_temp,
#             "Max Temp": region_max_temp,
#             "Min Temp": region_min_temp,
#             "Humidity": region_humid
#             })
            

#         except:
#             print(f"province {a} not found... Sorry!")
            
#             try:
                
#                 b = row["country"]
#                 query_url = (f"https://api.openweathermap.org/data/2.5/weather?q={b}&appid={weather_api_key}&units=imperial")
#                 response = requests.get(query_url).json()

#                 #get data for region 
#                 region_lng = response['coord']['lon']
#                 region_lat = response['coord']["lat"]
#                 region_temp= response['main']['temp']
#                 region_max_temp = response['main']['temp_max']
#                 region_min_temp = response['main']['temp_min']
#                 region_humid = response['main']['humidity']
                

#                 #Create dictionary with data gathered
#                 city_weather.append({
#                 "Region": i,
#                 "Lat": region_lat,
#                 "Lng": region_lng,
#                 "Temp": region_temp,
#                 "Max Temp": region_max_temp,
#                 "Min Temp": region_min_temp,
#                 "Humidity": region_humid
#                 })
                

#             except:      
#                 print(f"country {b} not found... Sorry!")
#                 pass
#     12
# print("Complete!")

In [ ]:
# weather = pd.DataFrame(city_weather)
#weather["diurnal range"] = cleaned_weather["Max Temp"] - cleaned_weather["Min Temp"]
#weather.to_csv("./Resources/output.csv", index_label = "City ID")
#weather

# Transform

In [3]:
#Begin cleaning by removing undesired columns and droping NaNs
cleaned_df = wine_df[["variety", "points", "price", "country", "province", "region_1"]]
cleaned_df = cleaned_df.dropna()
cleaned_df.head()


,variety,points,price,country,province,region_1
2,Pinot Gris,87,14.0,US,Oregon,Willamette Valley
3,Riesling,87,13.0,US,Michigan,Lake Michigan Shore
4,Pinot Noir,87,65.0,US,Oregon,Willamette Valley
5,Tempranillo-Merlot,87,15.0,Spain,Northern Spain,Navarra
6,Frappato,87,16.0,Italy,Sicily & Sardinia,Vittoria
...,...,...,...,...,...,...
129965,Pinot Gris,90,28.0,France,Alsace,Alsace
129967,Pinot Noir,90,75.0,US,Oregon,Oregon
129968,Gewürztraminer,90,30.0,France,Alsace,Alsace
129969,Pinot Gris,90,32.0,France,Alsace,Alsace


In [4]:
#Create seperate df for region data
province = cleaned_df[["country", "province", "region_1"]]
provinces_cleaned = province.drop_duplicates()
provinces_cleaned

,country,province,region_1
2,US,Oregon,Willamette Valley
3,US,Michigan,Lake Michigan Shore
5,Spain,Northern Spain,Navarra
6,Italy,Sicily & Sardinia,Vittoria
7,France,Alsace,Alsace
...,...,...,...
126567,US,California,Sonoma-Napa-Lake
126987,US,California,Sonoma County-Santa Barbara County
129331,Italy,Northeastern Italy,Del Veneto
129411,Italy,Veneto,Bardolino Superiore


In [20]:
#Read saved weather api data
output = "Resources/output.csv"
cleaned_weather = pd.read_csv(output)
cleaned_weather

,City ID,Region,Lat,Lng,Max Temp,Min Temp,Temp,Humidity,diurnal range
0,0,Willamette Valley,44.0001,-120.5014,60.03,55.22,55.24,39,4.81
1,1,Lake Michigan Shore,44.2503,-85.5003,71.67,68.50,71.49,83,3.17
2,2,Navarra,42.7500,-1.6667,53.53,51.12,51.13,93,2.41
3,3,Vittoria,36.9537,14.5332,73.92,73.92,73.92,78,0.00
4,4,Alsace,48.5000,7.5000,53.74,53.31,53.31,87,0.43
...,...,...,...,...,...,...,...,...,...
1201,1201,Sonoma-Napa-Lake,38.3004,-76.5074,73.08,62.42,68.72,73,10.66
1202,1202,Sonoma County-Santa Barbara County,38.3004,-76.5074,73.08,62.42,68.72,73,10.66
1203,1203,Del Veneto,32.1840,-96.8847,88.12,82.06,85.28,37,6.06
1204,1204,Bardolino Superiore,45.5000,11.7500,61.02,58.75,60.91,83,2.27


In [8]:
#Create unique region id 
provinces_cleaned['region_id'] = provinces_cleaned.groupby(['country','province', 'region_1']).ngroup()
provinces_cleaned = provinces_cleaned[["region_id", "country", "province", "region_1"]]
provinces_cleaned

<ipython-input-8-281dc46b42e2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  provinces_cleaned['region_id'] = provinces_cleaned.groupby(['country','province', 'region_1']).ngroup()


,region_id,country,province,region_1
2,1176,US,Oregon,Willamette Valley
3,1134,US,Michigan,Lake Michigan Shore
5,919,Spain,Northern Spain,Navarra
6,692,Italy,Sicily & Sardinia,Vittoria
7,117,France,Alsace,Alsace
...,...,...,...,...
126567,1104,US,California,Sonoma-Napa-Lake
126987,1099,US,California,Sonoma County-Santa Barbara County
129331,582,Italy,Northeastern Italy,Del Veneto
129411,809,Italy,Veneto,Bardolino Superiore


In [9]:
#Merged data to add region id to wine data
wine_data = pd.merge(provinces_cleaned, cleaned_df, on = "region_1")
clean_wine = wine_data[["region_id", "variety", "points", "price"]]
clean_wine

,region_id,variety,points,price
0,1176,Pinot Gris,87,14.0
1,1176,Pinot Noir,87,65.0
2,1176,Pinot Noir,86,22.0
3,1176,Pinot Noir,91,38.0
4,1176,Pinot Noir,85,28.0
...,...,...,...,...
101408,1104,Cabernet Sauvignon,91,20.0
101409,1099,Chardonnay,86,14.0
101410,582,Corvina,86,11.0
101411,809,Red Blend,89,15.0


In [10]:
#Rename region to be able to merge on. Then Merge Weather data with region to get the region id in the weather data
provinces_cleaned = provinces_cleaned.rename(columns={"region_1": "Region"})
cleaned = pd.merge(cleaned_weather, provinces_cleaned, on = "Region")
cleaned = cleaned[["region_id", "Lat", "Lng", "Max Temp", "Min Temp", "Temp", "Humidity", "diurnal range"]]
cleaned = cleaned.rename(columns={"Lat": "lat", "Lng": "lng", "Max Temp": "max_temp", "Min Temp": "min_temp", "Temp": "temp", "Humidity": "humidity", "diurnal range": "diurnal_range"}) 
cleaned

,region_id,lat,lng,max_temp,min_temp,temp,humidity,diurnal_range
0,1176,44.0001,-120.5014,60.03,55.22,55.24,39,4.81
1,1134,44.2503,-85.5003,71.67,68.50,71.49,83,3.17
2,919,42.7500,-1.6667,53.53,51.12,51.13,93,2.41
3,692,36.9537,14.5332,73.92,73.92,73.92,78,0.00
4,117,48.5000,7.5000,53.74,53.31,53.31,87,0.43
...,...,...,...,...,...,...,...,...
1205,1104,38.3004,-76.5074,73.08,62.42,68.72,73,10.66
1206,1099,38.3004,-76.5074,73.08,62.42,68.72,73,10.66
1207,582,32.1840,-96.8847,88.12,82.06,85.28,37,6.06
1208,809,45.5000,11.7500,61.02,58.75,60.91,83,2.27


# Load

In [14]:

protocol = "postgres"
user = "postgres"
location = "localhost"
port = "51070"
database = "etl"

connection_string = f"{user}:{db_pass}@{location}:{port}/{database}"
print(connection_string)

postgres:2933@localhost:51070/etl


In [21]:
engine = create_engine(f'postgresql://{connection_string}')

In [23]:
engine.table_names()

<ipython-input-23-d72bc75a6b62>:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 51070?
could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 51070?

(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [ ]:
provinces_cleaned.to_sql(name='region', con=engine, if_exists='append', index=True)

In [ ]:
cleaned.to_sql(name='weather', con=engine, if_exists='append', index=True)

In [ ]:
clean_wine.to_sql(name='wine', con=engine, if_exists='append', index=True)